In [1]:
#!pip install boto3

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import boto3
from io import BytesIO
import io
from sklearn.preprocessing import MinMaxScaler
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import pearsonr
import textdistance
import warnings

## Credentials for Amazon S3

In [6]:
aws_access_key_id = ''
aws_secret_access_key = ''

bucket_name = ''

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

## The list contains generic CS keywords

In [7]:
CS_keywords = set(['python', 'pytorch', 'sql', 'mxnet', 'mlflow', 'einstein', 'theano', 'pyspark', 'solr', 'mahout',
 'cassandra', 'aws', 'powerpoint', 'spark', 'pig', 'sas', 'java', 'nosql', 'docker', 'salesforce', 'scala', 'c++', 'net', 'tableau', 'pandas', 'scikitlearn', 'sklearn', 'matlab', 'scala', 'keras', 'tensorflow', 'clojure',
 'caffe', 'scipy', 'numpy', 'matplotlib', 'vba', 'spss', 'linux', 'azure', 'cloud', 'gcp', 'mongodb', 'mysql', 'oracle',
 'redshift', 'snowflake', 'kafka', 'javascript', 'qlik', 'jupyter', 'perl', 'bigquery', 'unix', 'react',
 'scikit', 'powerbi', 's3', 'ec2', 'lambda', 'ssrs', 'kubernetes', 'hana', 'spacy', 'tf', 'django', 'sagemaker',
 'seaborn', 'mllib', 'github', 'git', 'elasticsearch', 'splunk', 'airflow', 'looker', 'rapidminer', 'birt', 'pentaho',
'jquery', 'nodejs', 'd3', 'plotly', 'bokeh', 'xgboost', 'rstudio', 'shiny', 'dash', 'h20', 'h2o', 'hadoop', 'mapreduce',
 'hive', 'cognos', 'angular', 'nltk', 'flask', 'node', 'firebase', 'bigtable', 'rust', 'php', 'cntk', 'lightgbm',
 'kubeflow', 'rpython', 'unixlinux', 'postgressql', 'postgresql', 'postgres', 'hbase', 'dask', 'ruby', 'julia', 'tensor',
 'dplyr','ggplot2','esquisse','bioconductor','shiny','lubridate','knitr','mlr','quanteda','dt','rcrawler','caret','rmarkdown',
 'leaflet','janitor','ggvis','plotly','rcharts','rbokeh','broom','stringr','magrittr','slidify','rvest',
 'rmysql','rsqlite','prophet','glmnet','text2vec','snowballc','quantmod','rstan','swirl','datasciencer', 
 'amazon web services', 'google cloud', 'sql server',
  'cleansing', 'chatbot', 'cleaning', 'blockchain', 'causality', 'correlation', 'bandit', 'anomaly', 'kpi',
 'dashboard', 'geospatial', 'ocr',  'pca', 'gis', 'svm', 'svd', 'tuning', 'hyperparameter', 'hypothesis',
 'salesforcecom', 'segmentation', 'biostatistics', 'unsupervised', 'supervised', 'exploratory',
 'recommender', 'recommendations', 'research', 'sequencing', 'probability', 'reinforcement', 'graph', 'bioinformatics',
  'knn', 'etl', 'normalization', 'classification', 'optimizing', 'prediction', 'forecasting',
 'clustering', 'optimization', 'visualization', 'nlp', 'c#',
 'regression', 'logistic', 'cnn', 'glm',
 'rnn', 'lstm', 'gbm', 'boosting', 'recurrent', 'convolutional', 'bayesian',
 'bayes', 'random forest', 'natural language processing', 'machine learning', 'decision tree', 'deep learning', 'experimental design',
 'time series', 'nearest neighbors', 'neural network', 'support vector machine', 'computer vision', 'machine vision', 'dimensionality reduction',
 'text analytics',  'a/b testing', 'data mining', 'kajsadouas', 'senior','intern', 'Data', ])


## Get the list of unique keywords that appear in the description

In [8]:
file_key = 'merged_dataset.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_key)
content = obj['Body'].read()
merged_df = pd.read_csv(BytesIO(content), engine='python')
display(merged_df.head(3))

,Title,Description,Company Name,City,State,Salary,Year,Month,Day,CS_keywords
0,Data Scientist - Cross Asset Desk Strategist T...,Data Scientist - Cross Asset Desk Strategist T...,Morgan Stanley,New York,New York,90000,2019.0,8.0,20.0,2
1,Senior Data Scientist - Infectious Disease and...,Senior Data Scientist - Infectious Disease and...,Carolinas HealthCare System,Charlotte,North Carolina,125000,2019.0,9.0,6.0,5
2,"Senior Data Scientist, Advanced Marketing Anal...","Senior Data Scientist, Advanced Marketing Anal...",Spotify,New York,New York,125000,2019.0,8.0,23.0,11


In [9]:
# Suppress PerformanceWarning
warnings.filterwarnings("ignore", message="DataFrame is highly fragmented.*")

# Tokenize the words and remove stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

merged_df['Description'] = merged_df['Description'].apply(lambda x: ' '.join([word.capitalize() for word in word_tokenize(x) if word.lower() not in stop_words]))

# Count word frequencies
word_frequencies = {}
for description in merged_df['Description']:
    words = word_tokenize(description)
    for word in words:
        word = word.lower()  # Convert to lowercase for consistency
        if word not in word_frequencies:
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

# Filter out words with frequency above 50
frequent_words = {word: freq for word, freq in word_frequencies.items() if freq > 50}

# Create bag-of-words representation
vectorizer = CountVectorizer(vocabulary=frequent_words.keys())
X = vectorizer.fit_transform(merged_df['Description'])

# Calculate correlation
correlations = {}
for word in frequent_words:
    idx = vectorizer.vocabulary_.get(word)
    if idx is not None:
        word_vector = X[:, idx].toarray().flatten()
        correlation, _ = pearsonr(word_vector, merged_df['Salary'])
        if not pd.isnull(correlation):  # Exclude words with NaN correlation
            correlations[word] = abs(correlation)

# Find redundant words based on Jaro-Winkler similarity and keep only the one with the highest correlation
threshold = 0.7 # You can adjust this threshold as needed
redundant_words = set()
for word1 in correlations:
    for word2 in correlations:
        if word1 != word2 and textdistance.jaro_winkler.similarity(word1, word2) > threshold:
            if correlations[word1] > correlations[word2]:
                redundant_words.add(word2)
            else:
                redundant_words.add(word1)

# Remove redundant words from frequent_words
frequent_words = {word: freq for word, freq in frequent_words.items() if word not in redundant_words}

# Sort the words based on their correlation with the salary column and remove redundant words
sorted_correlations = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
sorted_correlations = [(word, corr) for word, corr in sorted_correlations if word not in redundant_words]

#print("Words with frequency above 50 and not redundant:", len(frequent_words))
print("Words with highest correlation with salary:", sorted_correlations)

[nltk_data] Downloading package punkt to /home/jeremy/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jeremy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/tmp/ipykernel_63626/948814392.py:35: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, _ = pearsonr(word_vector, merged_df['Salary'])


Words with highest correlation with salary: [('python', 0.19886745519756654), ('work', 0.19582876337073574), ('data', 0.19295753214710162), ('experience', 0.17431311358590212), ('scientist', 0.17268792569686944), ('new', 0.15354276957001775), ('opportunities', 0.15019754403229654), ('insights', 0.14787489406646623), ('skills', 0.1468227181885829), ('problems', 0.1397344220184944), ('communication', 0.13341254386076665), ('help', 0.13216785971370518), ('build', 0.1311344348955121), ('programming', 0.12954808497310677), ('team', 0.12875060880594835), ('drive', 0.12868237014977418), ('including', 0.12748442462796286), ('passion', 0.12350654759714375), ('business', 0.12331215076141142), ('responsibilities', 0.12300449949198337), ('statistics', 0.12196279486537379), ('sql', 0.12170040785798925), ('key', 0.11976724543630891), ('ca', 0.11853650850023585), ('actionable', 0.11781130547401442), ('ability', 0.11693902421646957), ('learn', 0.1168654047039393), ('analyses', 0.11655381756896774), ('

## Adding top 30 correlated keyword columns in the merged dataset

In [10]:
# Add frequency columns for top 30 correlated keywords
for word, _ in sorted_correlations[:30]:
    merged_df[word] = merged_df['Description'].apply(lambda x: sum(textdistance.jaro_winkler.similarity(word, w) > threshold for w in word_tokenize(x)))

In [11]:
print(merged_df.columns)

Index(['Title', 'Description', 'Company Name', 'City', 'State', 'Salary',
       'Year', 'Month', 'Day', 'CS_keywords', 'python', 'work', 'data',
       'experience', 'scientist', 'new', 'opportunities', 'insights', 'skills',
       'problems', 'communication', 'help', 'build', 'programming', 'team',
       'drive', 'including', 'passion', 'business', 'responsibilities',
       'statistics', 'sql', 'key', 'ca', 'actionable', 'ability', 'learn',
       'analyses', 'open', 'environment'],
      dtype='object')


## Validating correlations

In [12]:
merged_df_corr = merged_df.drop(['Description', 'Title', 'Company Name', 'City', 'State'], axis=1)

salary_column = merged_df_corr['Salary']
correlations = merged_df_corr.corrwith(salary_column)


sorted_correlations = correlations.abs().sort_values(ascending=False)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(sorted_correlations)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Salary              1.000000
work                0.221658
data                0.194326
python              0.193922
experience          0.189389
including           0.185625
CS_keywords         0.183533
communication       0.174846
actionable          0.174808
open                0.171710
problems            0.170969
analyses            0.169708
drive               0.168311
new                 0.168280
team                0.165778
insights            0.163176
scientist           0.163092
learn               0.160551
passion             0.157631
help                0.153462
skills              0.148021
build               0.142170
ability             0.139136
business            0.137521
ca                  0.136612
statistics          0.134026
sql                 0.128271
opportunities       0.125180
responsibilities    0.124953
key                 0.124281
programming         0.111108
environment         0.108739
Month               0.022666
Day                 0.020770
Year          

/home/jeremy/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jeremy/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [13]:
merged_df.head(2)

,Title,Description,Company Name,City,State,Salary,Year,Month,Day,CS_keywords,...,statistics,sql,key,ca,actionable,ability,learn,analyses,open,environment
0,Data Scientist - Cross Asset Desk Strategist T...,Data Scientist - Cross Asset Desk Strategist T...,Morgan Stanley,New York,New York,90000,2019.0,8.0,20.0,2,...,3,0,0,3,0,2,5,8,0,1
1,Senior Data Scientist - Infectious Disease and...,Senior Data Scientist - Infectious Disease Inf...,Carolinas HealthCare System,Charlotte,North Carolina,125000,2019.0,9.0,6.0,5,...,4,0,1,13,8,7,5,7,4,5


## Push the dataset to Amazon S3 datalake

In [14]:
output_file_key = 'with_20_keywords.csv'

csv_buffer = io.StringIO()
merged_df.to_csv(csv_buffer, index=False)

s3.put_object(Body = csv_buffer.getvalue(), Bucket = bucket_name, Key=output_file_key)

{'ResponseMetadata': {'RequestId': 'XE378TY1B4VZDNKX',
  'HostId': 'LUJsb6SgJhxeIaHbaJLhbvPg70kKUQKgg3FpCFs6iLSWFkEn7DJQckD5cVsOITtYj/zqmq9wwWt81wNa+uAe8AyGP5dCJLNs',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LUJsb6SgJhxeIaHbaJLhbvPg70kKUQKgg3FpCFs6iLSWFkEn7DJQckD5cVsOITtYj/zqmq9wwWt81wNa+uAe8AyGP5dCJLNs',
   'x-amz-request-id': 'XE378TY1B4VZDNKX',
   'date': 'Mon, 15 Apr 2024 18:23:36 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f330a8478c664648d35e8954c07dc69c"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f330a8478c664648d35e8954c07dc69c"',
 'ServerSideEncryption': 'AES256'}

### Normalization

#### Before normalization:

In [ ]:
print(final_df['Salary'])

In [ ]:
scaler = MinMaxScaler()
final_df.loc[:, 'Salary'] = scaler.fit_transform(final_df['Salary'].values.reshape(-1, 1))
# If we want to reverse the normalization
# final_df.loc[:, 'Salary'] = scaler.inverse_transform(final_df['Salary'].values.reshape(-1, 1))

#### After normalization:

In [ ]:
print(final_df['Salary'])